In [3]:
"""
A collection of functions to perform a dark photon search on an averaged spectrum


Functions
-------
median_filt(
    
):
    LPF for spectrum. Recomended as a pre-filter if
    spec contains large, very narrow candidates

butter_filt(

):
    LPF for spectrum. 

SG_filt(

):
    LPF for spectrum

match_filt(
    
):
    Performs a matched filtering on approx. normally distributed data
    Internal helper functions include freq2idx and genNuSteps

set_limit(

):
    Sets a limit (or series of limits) on an array of 
    normally distributed data (or list of arrays of normal data)

get_required_sigma(

):
    Given a number of samples and a desired confidence limit, tells you how 
    many "N" sigma you need to go to in order to be CL% confident no samples
    are above the calculated "N" 
"""
import numpy as np 
import scipy.signal as spsig
import scipy.ndimage as spim
import cupyx.scipy.ndimage as cpim
import cupy as cp
from scipy.signal import butter, filtfilt
import scipy.stats
import scipy.special
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib widget


def median_filt(spec, filter_size=50, gpuFlag = 1):
    """
    Function to apply median filtering to a spectrum.

    Parameters
    ----------
    spectrum : array
        The input power spectrum.

    filter_size : int, optional
        Size of the median filter. Default is 50.

    Returns
    -------
    spectrum_filtered : array
        The median-filtered spectrum.
    """
    if gpuFlag:
        spectrum_gpu = cp.array(spec)
        spectrum_filtered_gpu = cpim.median_filter(
            spectrum_gpu,
            size=(filter_size),
            origin=0
        )
        spectrum_filtered = spectrum_filtered_gpu.get()
    else:
        spectrum_filtered = spim.median_filter(
            spec,
            size=(filter_size),
            origin=0
        )
    return spectrum_filtered

def butter_filt(spec, cutoff=40000, order=6):
    """
    Function to apply Butterworth low-pass filtering to a spectrum.

    Parameters
    ----------
    spectrum : array
        The input power spectrum.

    cutoff : int, optional
        Cutoff frequency for the filter. Default is 40000.

    order : int, optional
        Order of the filter. Default is 6.

    Returns
    -------
    spectrum_filtered : array
        The Butterworth low-pass filtered spectrum.
    """
    nyq = 0.5 * len(spec)  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    spectrum_filtered = filtfilt(b, a, spec)
    return spectrum_filtered

def sg_filt(spec, window_length=51, polyorder=4):
    """
    Apply a Savitzky-Golay filter to the spectrum.

    Parameters
    ----------
    window_length : int, optional
        Length of the filter window; must be a positive odd integer, default is 51.
    polyorder : int, optional
        Order of the polynomial used to fit the samples, default is 1.

    Returns
    -------
    arr
        Filtered spectrum.
    """
    spectrum_filtered = spsig.savgol_filter(spec, window_length=window_length, polyorder=polyorder)
    return spectrum_filtered

def matched_filt(freqs, 
    spec, 
    startFreq, 
    stopFreq, 
    percent, 
    overlap
    ):
    """
    Apply a matched filter to the input spectrum over a specified frequency range.

    This function divides the freqs range into steps and applies a convolution
    operation with a lambda template for each step. The function returns the 
    convolved output for each freqs segment.

    Parameters
    ----------
    freqs : array-like
        Array of frequencies corresponding to the input spectrum.
    spec : array-like
        Array of spectral values to be filtered.
    startFreq : float
        Starting freqs of the range over which the filter is applied.
    stopFreq : float
        Stopping frequency of the range over which the filter is applied.
    percent : float
        Percentage to determine the step size between successive frequencies in the
        range (startFreq to stopFreq).
    overlap : int
        Number of overlapping elements between successive frequency segments.

    Returns
    -------
    tuple
        A tuple containing two lists:
        - outputFreqs: List of arrays with the frequencies corresponding to each segment.
        - outputConv: List of arrays with the convolved values for each segment.

    Notes
    -----
    Frequency step generation (`gen_nu_steps`) is
    defined internally within this function.
    """
    def gen_nu_steps(freqs, startFreq, stopFreq, percent):
        freq    = startFreq
        nuStepList  = []
        while freq < stopFreq:
            #if freq > freqs:
            #    nu0Arr.append(400)
            nuStepList.append(freq)
            endFreq = freq + freq*percent/100
            freq = endFreq
        nuStepList.append(stopFreq) 
        return nuStepList
        
    nuStepList  = gen_nu_steps(freqs, startFreq, stopFreq, percent)
    outputConv  = []
    outputFreqs = []
    #change template over frequency range
    for i, nu in enumerate(nuStepList[0:-1]):
        idxStart        = freq2idx(nuStepList[i], freqs) - overlap
        nu0             = nuStepList[i] + (nuStepList[i+1] - nuStepList[i])/2
        if 0: #nuStepList[i+1] == stopFreq:
            idxStop     = freq2idx(nuStepList[i+1], freqs) - 10
        else:
            idxStop     = freq2idx(nuStepList[i+1], freqs) + overlap
        lamOut      = lam(freqs, nu0, p0=1, nBins=100)
        subConv     = np.convolve(np.flip(lamOut[1]), spec[idxStart:idxStop], mode='same')[overlap:-overlap]
        subFreqs    = freqs[idxStart+overlap:idxStop-overlap]
        
        outputConv.append(subConv)
        outputFreqs.append(subFreqs)
    return (outputFreqs, outputConv)

def lam(nu, nu0, p0 = None, nBins=None, v0=220, vlab=233, offset = 0):
    """
    Lambda function for matched filtering.
    Work in MHz, this will explode for large frequency values (ie 100e6 Hz)
    velocity args in km/s
    vlab has yearly modulation between about 220 and 255, see fig 3a in Gramolin
    output is normilized to p0 if given

    Parameters
    ----------
    nu : float
        Frequency.
    nu0 : float
        Center frequency.
    p0 : float, optional
        Power normalization value, default is None.
    nBins : int, optional
        Number of bins, default is None.
    v0 : int, optional
        Velocity parameter, default is 220.
    vlab : int, optional
        Lab velocity parameter, default is 233.
    offset : int, optional
        Frequency offset, default is 0.

    Returns
    -------
    tuple
        Frequencies and output power spectral density.
    """
    if nBins is not None:
        # find the index of nu0 in nu array
        idx = np.abs(nu-nu0).argmin()

        # calculate the slice start and end indices
        start_idx = max(0, idx - nBins//2)
        end_idx = min(len(nu), idx + nBins//2)
        nu = nu[start_idx:end_idx]

    c       = 299792 #km/s
    nu0     = nu0 + offset
    beta    = np.zeros_like(nu) # initialise beta with zeros

    mask    = (nu > nu0) # create a mask of values where nu > nu0
    # calculate beta only for values where nu > nu0
    beta[mask] = (2*c*vlab)/(v0**2) * np.sqrt((2*(nu[mask]-nu0))/(nu0))
    beta    = np.minimum(beta, 100)
    a       = (2*c**2)/(np.sqrt(np.pi)*v0*vlab*nu0)
    b       = -((beta**2*v0**2)/(4*vlab**2)) - (vlab**2/v0**2) 

    lamArr  = a*np.exp(b)*np.sinh(beta)
    if p0 is not None:
        outPSD = p0/np.sum(lamArr) * lamArr #normilize to p0
    else:
        outPSD = lamArr

    return nu, outPSD 

def get_required_sigma(nBins, desired_confidence):    
    """
    Calculate the required standard deviation (sigma) level to achieve a specified
    confidence level that none of the samples in a given number of bins will exceed
    that sigma level.

    The function solves for the sigma level using the cumulative distribution function
    of the normal distribution and the binomial probability for the given number of bins.

    Parameters
    ----------
    nBins : int
        The number of bins or samples drawn from a normal distribution.
    desired_confidence : float
        The desired confidence level (between 0 and 1) for the probability that none
        of the samples will exceed the calculated sigma level. eg 0.95

    Returns
    -------
    float
        The calculated sigma level required to achieve the desired confidence.

    Notes
    -----
    The function uses the `fsolve` method from SciPy's optimize module to solve for
    the sigma value. If the solver does not converge, it returns a guessed value
    based on the inverse error function.
    """

    desired_confidence = 1-desired_confidence
    def probability_greater_than(sigma):
        return 0.5-scipy.special.erf(sigma/np.sqrt(2))/2

    def binomial(N, num_successes, prob_of_success):
        return scipy.special.comb(N, num_successes)*prob_of_success**(num_successes)*(1-prob_of_success)**(N-num_successes)

    def probability_none_above(nBins, sigma):
        # returns probability that after nBins of draws from a normal distribution
        # that none of the draws will be above the sigma upper limit (mean+sigma)
        return 1-binomial(nBins, nBins, 1-probability_greater_than(sigma))

    guess     = scipy.special.erfinv(1-(2*(desired_confidence)/nBins))*np.sqrt(2) #from taylor expantion
    nSig, solDict, itr, msg   = fsolve(lambda sigma: probability_none_above(nBins, sigma)-desired_confidence, guess, full_output=True)
    nSig = nSig[0]
    if itr != 1:
        print('fSolve in get_required_sigma failed. Using guess')
        print(msg)
        nSig = guess
    
    return nSig

def set_lim(freqs, spec, CL):
    """
    Calculate the frequency limits and standard deviations for the given spectra.

    This function computes the limits by applying the required sigma to each segment
    of the input spectra and then concatenates the results to form the complete limit.
    It also flattens and concatenates the input frequency and spectrum arrays to match
    the shape of the limit.

    Parameters
    ----------
    freqs : arr or list of arr
        Array(s) of frequencies corresponding to the input spectra. If a single array
        is provided, it is wrapped in a list to form an array of arrays.
    spec : arr or list of arr
        Array(s) of spectra to calculate the limits for. If a single array is provided,
        it is wrapped in a list to form an array of arrays.
    CL : float
        Confidence level for calculating the required sigma. This value is used to
        determine how many standard deviations to apply to the spectra.

    Returns
    -------
    tuple
        A tuple containing three arrays: the concatenated frequency array, the
        concatenated spectrum array, and the calculated limit array.
    """
    # If the inputs are 1-D arrays, wrap them in a list to make them an array of arrays
    if isinstance(spec, (np.ndarray, list)) and not isinstance(spec[0], (np.ndarray, list)):
        freqs   = [freqs]
        spec    = [spec]
    nSamps = len(np.hstack(freqs))
    nSig = get_required_sigma(nSamps, CL)

    limits = []
    for idx in range(len(spec)):
        limits.append(nSig * 1.4826 * scipy.stats.median_abs_deviation(spec[idx]))

    limit   = np.concatenate([np.repeat(limit, len(freqs)) for limit, freqs in zip(limits, freqs)])
    freqs   = np.hstack(freqs)
    spec    = np.hstack(spec)

    return (freqs, spec, limit)

def freq2idx(freq, freqs):
    return int(freq/freqs[-1]*len(freqs))


In [1]:
# analysis function

def analysis(freqs,
    spec,
    freqStart,
    freqStop,
    CL,
    medFiltBins,
    butterFreq,
    butterOrder,
    percent,
    overlap,
    stepsize = 1000e-6
    ):
    idxStart    = freq2idx(freqStart, freqs)
    idxStop     = freq2idx(freqStop, freqs)
    
    # Fit H to spec
    medianFilt = median_filt(spec, filter_size=medFiltBins)
    H = (butter_filt(medianFilt, cutoff = butterFreq, order = butterOrder))

    #Norm spec: mean = 1, STD = 1
    normSpec    = spec / H




    if 1: #matched filter
        matchedOut      = matched_filt(freqs=freqs,
            spec=normSpec,
            startFreq=freqStart,
            stopFreq=freqStop,
            percent=percent,
            overlap=overlap)
        matchedFreqs    = matchedOut[0]
        matchedSpec     = matchedOut[1]

        nSigmaTup       = set_lim(matchedFreqs, matchedSpec, CL)
        matchedFreqs    = nSigmaTup[0]
        matchedSpec     = nSigmaTup[1]
        matchedLim      = nSigmaTup[2]
        
        rawCanList  = freqs[idxStart:idxStop][((matchedSpec - 1) > matchedLim)]

    else: #dont matched filter    
        nSigmaTup   = set_lim(freqs, normSpec, 0.95)
        nSigmaLim   = nSigmaTup[2]

        #nSigma      = get_required_sigma((idxStop - idxStart), 0.95)
        #oneSigMad   = (1.4826*scipy.stats.median_abs_deviation(normSpec))
        nSigmaTup       = set_lim(freqs, normSpec, CL)
        lim      = nSigmaTup[2]
        print(len(freqs[idxStart:idxStop]))
        rawCanList  = freqs[idxStart:idxStop][((normSpec[idxStart:idxStop] - 1) > nSigmaLim)]

    rawCanList = np.asarray(rawCanList)
    splitList =  np.split(rawCanList, np.where(np.diff(rawCanList) > stepsize)[0]+1)
    if len(rawCanList) == 0:
        return(np.asarray([]))
    canList = [can[0] for can in splitList]

    return canList


In [4]:
#fake data

def genSpec(freqs, H, data_mean, data_std, nu0, sigPow, offset=0):
    """
    Generate a fake spectrum with dark photon signal injected

    Parameters
    ----------
    freqs : array-like
        Array of frequencies corresponding to the input spectrum.
    H : array-like
        Array of H extracted from data set (or generated)
    data_mean : float
        Mean of spectrum that generated H (or arb)
    data_std : float
        Mean of spectrum that generated H (or arb)
    nu0 : float
        frequency to inject signal in MHz
    sigPow : float
        Power of injected signal in units of data_std
    offset : float
        "jitter" the placement of fake signal
    

    Returns
    -------
    array
        fake spectrum
    """
    nu0Idx      = freq2idx(nu0,freqs)
    noise       = np.random.normal(data_mean, data_std, len(freqs))
    sig         = lam(freqs, nu0, p0=data_std*sigPow, nBins=None)[1]
    return  H * (noise + sig)

#spec params
freqs       = np.linspace(0,400, 2**23)
freqStart   = 49
freqStop    = 299.9
idxStart    = freq2idx(freqStart, freqs)
idxStop     = freq2idx(freqStop, freqs)

HData       = np.load('H_med51_butter40000_order6_from_Run1p4_avgAll_11_20_23_gitignore.npy')
#mean and STD extracted from real data in other notebook
data_mean  = 0.999987900
data_std   = 0.000173962 

#remeasure 11_20_23
data_mean  = 0.9999932448
data_std   = 0.0001741190

fakeSpec = genSpec(freqs, HData, data_mean, data_std, nu0 = 42, sigPow = 15, offset=0 )
realSpec    = np.load('../run1p4_analysis/avgSpecs_gitignore/avgAntSpec_WALL_33p6M_FFTs_11_16_23.npy')[1:]

plt.close('all')
plt.figure()
plt.plot(freqs, fakeSpec, label='fake spec')
plt.plot(freqs, realSpec, label ='real spec', alpha = 0.5)
plt.legend()
plt.show()


# calculate common bin edges
min_bin_edge = min(np.min((fakeSpec/HData)[idxStart:idxStop]), np.min((realSpec/HData)[idxStart:idxStop]))
max_bin_edge = max(np.max(((fakeSpec/HData)[idxStart:idxStop])), np.max((realSpec/HData)[idxStart:idxStop]))

bin_edges = np.linspace(min_bin_edge, max_bin_edge, 500)  # this creates 50 bins

# plot histograms
plt.figure(figsize=(10, 5))
plt.title('Z score')
plt.hist(((fakeSpec/HData)[idxStart:idxStop]), bins=bin_edges, label='fake spec')
plt.hist(((realSpec/HData)[idxStart:idxStop]), bins=bin_edges, label='Real spec', alpha = 0.5)
plt.legend(loc='upper left')
plt.semilogy()
plt.show()

In [ ]:
#analyis

#analysis params
medFiltBins = 51
butterFreq  = 40000
butterOrder = 6
startFreq   = 44    # MHz
stopFreq    = 306   # MHz
percent     = 10    # matched filter
overlap     = 100   # matched filter, bins
CL          = 0.95  # confidence limit

#fake spec params
freqs       = np.linspace(0,400, 2**23)
freqStart   = 44
freqStop    = 306
HData       = np.load('H_med51_butter40000_order6_from_Run1p4_avgAll_11_20_23_gitignore.npy')
#mean and STD extracted from real data in other notebook
data_mean  = 0.9999932448
data_std   = 0.0001741190
nu0         = 294
sigPow      = 20
offset      = 0




In [5]:
#get_required_sigma(freq2idx(freqStop, freqs) -freq2idx(freqStart, freqs), .68)
get_required_sigma(1, .68)

0.4676987991145085

In [28]:

offset      = np.random.uniform(-25e-6, 25e-6) #jitter the fake signal by +- half a bin
fakeSpec    = genSpec(freqs, HData, data_mean, data_std, nu0, sigPow, offset)


canList = analysis(freqs,fakeSpec,medFiltBins=medFiltBins,butterFreq=butterFreq,butterOrder=butterOrder,freqStart=startFreq,freqStop=stopFreq,percent=percent,overlap=overlap,CL = CL)

In [29]:
spec        = fakeSpec
stepsize    = 100

idxStart    = freq2idx(freqStart, freqs)
idxStop     = freq2idx(freqStop, freqs)

# Fit H to spec
medianFilt = median_filt(spec, filter_size=medFiltBins) #120ms
H = (butter_filt(medianFilt, cutoff = butterFreq, order = butterOrder)) #124ms

#Norm spec: mean = 1, STD = 1
normSpec    = spec / H

nSigmaTup   = set_lim(freqs, normSpec, 0.95) #244ms
nSigmaLim   = nSigmaTup[2]

nSigma      = get_required_sigma((idxStop - idxStart), 0.95)
oneSigMad   = (1.4826*scipy.stats.median_abs_deviation(normSpec))



matchedOut      = matched_filt(freqs=freqs,
    spec=normSpec,
    startFreq=freqStart,
    stopFreq=freqStop,
    percent=percent,
    overlap=overlap)
matchedFreqs    = matchedOut[0]
matchedSpec     = matchedOut[1]

nSigmaTup       = set_lim(matchedFreqs, matchedSpec, CL)
matchedFreqs    = nSigmaTup[0]
matchedSpec     = nSigmaTup[1]
matchedLim      = nSigmaTup[2]

rawCanList  = freqs[idxStart:idxStop][((matchedSpec - 1) > matchedLim)]
rawCanList = np.asarray(rawCanList)
splitList =  np.split(rawCanList, np.where(np.diff(rawCanList) > stepsize)[0]+1)
canList = [can[0] for can in splitList]


In [53]:
nu0List         = np.linspace(45, 46, 1)
sigPowList      = np.linspace(5, 10, 5) 


nTrailsPerStep  = 20

truePosFracArr      = np.zeros((len(sigPowList), len(nu0List)))
falsePosFracArr     = np.zeros((len(sigPowList), len(nu0List)))

In [19]:
#MC
for nuIdx, nu0 in enumerate(nu0List):
    for powIdx, sigPow in enumerate(sigPowList):
        truePosCount    = 0
        falsePosCount   = 0
        print(f'nu0, sigPow :  {nu0, sigPow}' )
        for trial in range(nTrailsPerStep):
            offset = np.random.uniform(-25e-6, 25e-6)

            nu0Idx      = freq2idx(nu0, freqs)
            fakeSpec    = genSpec(freqs, HData, data_mean, data_std, nu0, sigPow,offset = offset)
            canList = analysis(freqs,
                fakeSpec,
                medFiltBins=medFiltBins,
                butterFreq=butterFreq,
                butterOrder=butterOrder,
                freqStart=startFreq,
                freqStop=stopFreq,
                percent=percent,
                overlap=overlap,
                CL = CL)
            for can in canList:
                #find real signals
                if abs(can - nu0) <= 1000e-6:
                    truePosCount += 1
                else:
                    falsePosArr += 1
            truePosArr[powIdx, nuIdx] = truePosCount
            falsePosArr[powIdx, nuIdx] = falsePosCount

nu0, sigPow :  (45.0, 5.0)


NameError: name 'truePosArr' is not defined

In [54]:
from concurrent.futures import ThreadPoolExecutor

def run_trial(nu0, sigPow, trial_params):
    offset      = np.random.uniform(-25e-6, 25e-6) #jitter the fake signal by +- half a bin
    fakeSpec    = genSpec(freqs, HData, data_mean, data_std, nu0, sigPow, offset)

    canList     = analysis(freqs,
                       fakeSpec,
                       **trial_params)

    truePosCount = 0
    falsePosCount = 0
    for can in canList: #check if candidate is within 1kHz of injection
        if abs(can - nu0) <= 1000e-6:
            truePosCount += 1
        else:
            falsePosCount += 1

    return truePosCount, falsePosCount

trial_params = {
    'medFiltBins': medFiltBins,
    'butterFreq': butterFreq,
    'butterOrder': butterOrder,
    'freqStart': startFreq,
    'freqStop': stopFreq,
    'percent': percent,
    'overlap': overlap,
    'CL': CL
}

for nuIdx, nu0 in enumerate(nu0List):
    for powIdx, sigPow in enumerate(sigPowList):
        print(f'nu0, sigPow :  {nu0, sigPow}' )
        #make more density around diagional
        #nTrials = int((1-(abs(nuIdx-powIdx)/10)) * nTrailsPerStep) + 1
        #print(nuIdx,powIdx)
        #print(nTrials)
        nTrials = nTrailsPerStep
        with ThreadPoolExecutor(max_workers=40) as executor:

            results = [executor.submit(run_trial, nu0, sigPow, trial_params) for _ in range(nTrials)]
            truePosCount = sum(result.result()[0] for result in results)
            falsePosCount = sum(result.result()[1] for result in results)

            truePosFrac = truePosCount/nTrials
            falsePosFrac = falsePosCount/nTrials
        
        truePosFracArr[powIdx, nuIdx] = truePosFrac
        falsePosFracArr[powIdx, nuIdx] = falsePosFrac
#np.save('data/truePosFracArr_sigPow5_25STD_freq45_305MHz_8_13_23asdf)',truePosFracArr)

nu0, sigPow :  (45.0, 5.0)
nu0, sigPow :  (45.0, 6.25)
nu0, sigPow :  (45.0, 7.5)
nu0, sigPow :  (45.0, 8.75)
nu0, sigPow :  (45.0, 10.0)


In [44]:
falsePosFracArr.mean()

0.0

In [45]:
truePosFracArr

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [ ]:
#same as above but try to limit frequency span

def run_trial(nu0, sigPow, trial_params):
    nu0Idx      = freq2idx(nu0, freqs)
    offset      = np.random.uniform(-25e-6, 25e-6) #jitter the fake signal by +- half a bin
    fakeSpec    = genSpec(freqs[nu0Idx-1000:nu0Idx+1000], HData[nu0Idx-1000:nu0Idx+1000], data_mean, data_std, nu0, sigPow, offset)
    

    canList     = analysis(freqs,
                       fakeSpec,
                       **trial_params)

    truePosCount = 0
    falsePosCount = 0
    for can in canList: #check if candidate is within 1kHz of injection
        if abs(can - nu0) <= 1000e-6:
            truePosCount += 1
        else:
            falsePosCount += 1

    return truePosCount, falsePosCount

trial_params = {
    'medFiltBins': medFiltBins,
    'butterFreq': butterFreq,
    'butterOrder': butterOrder,
    'freqStart': startFreq,
    'freqStop': stopFreq,
    'percent': percent,
    'overlap': overlap,
    'CL': CL
}

for nuIdx, nu0 in enumerate(nu0List):
    for powIdx, sigPow in enumerate(sigPowList):
        print(f'nu0, sigPow :  {nu0, sigPow}' )
        #make more density around diagional
        #nTrials = int((1-(abs(nuIdx-powIdx)/10)) * nTrailsPerStep) + 1
        #print(nuIdx,powIdx)
        #print(nTrials)
        nTrials = nTrailsPerStep
        with ThreadPoolExecutor(max_workers=40) as executor:

            results = [executor.submit(run_trial, nu0, sigPow, trial_params) for _ in range(nTrials)]
            truePosCount = sum(result.result()[0] for result in results)
            falsePosCount = sum(result.result()[1] for result in results)

            truePosFrac = truePosCount/nTrials
            falsePosFrac = falsePosCount/nTrials
        
        truePosFracArr[powIdx, nuIdx] = truePosFrac
        falsePosFracArr[powIdx, nuIdx] = falsePosFrac
#np.save('data/truePosFracArr_sigPow7_23STD_freq49_299MHz_8_10_23',truePosFracArr)

In [ ]:
from multiprocessing import Pool

def run_trial(params):
    nu0, sigPow, trial_params = params
    offset      = np.random.uniform(-25e-6, 25e-6)
    nu0Idx      = freq2idx(nu0, freqs)
    fakeSpec    = genSpec(freqs, HData, data_mean, data_std, nu0, sigPow, offset)[nu0Idx-2000:nu0Idx+2000]
    canList = analysis(freqs,
                       fakeSpec,
                       **trial_params)

    truePosCount = 0
    falsePosCount = 0
    for can in canList:
        if abs(can - nu0) <= 1000e-6:
            truePosCount += 1
        else:
            falsePosCount += 1

    return truePosCount, falsePosCount

trial_params = {
    'medFiltBins': medFiltBins,
    'butterFreq': butterFreq,
    'butterOrder': butterOrder,
    'freqStart': startFreq,
    'freqStop': stopFreq,
    'percent': percent,
    'overlap': overlap,
    'CL': CL
}

for nuIdx, nu0 in enumerate(nu0List):
    for powIdx, sigPow in enumerate(sigPowList):
        print(f'nu0, sigPow :  {nu0, sigPow}' )
        with Pool() as pool:
            results = pool.map(run_trial, [(nu0, sigPow, trial_params) for _ in range(nTrailsPerStep)])
            truePosCount = sum(result[0] for result in results)
            falsePosCount = sum(result[1] for result in results)
            
        truePosArr[powIdx, nuIdx] = truePosCount
        falsePosArr[powIdx, nuIdx] = falsePosCount


In [ ]:
plt.close('all')

#truePosFracArr  = np.load('data/truePosFracArr_sigPow7_23STD_freq49_299MHz_8_10_23.npy')

plt.figure(figsize=(10,5))
plt.tight_layout()
plt.title('Fraction of False Positives (Analysis pipeline 8/10/23)')
plt.yticks(np.arange(len(sigPowList)), [round(sigPow,2) for sigPow in np.flip(sigPowList)])
plt.xticks(np.arange(len(nu0List)), [round(nu0,1) for nu0 in nu0List])
plt.imshow(np.flip(falsePosFracArr, axis=0))
plt.ylabel('Signal Power (Normilized to 1 STD)')
plt.xlabel('Frequecy (MHz)')

# Overlay temperature values
def custom_format(value):
    formatted_value = f'{value:.2f}'
    if formatted_value == '1.00':
        return '1'
    if formatted_value == '0.00':
        return '0'
    if formatted_value.startswith('0'):
        return formatted_value[1:]

    return formatted_value
    

for i in range(len(sigPowList)):
    for j in range(len(nu0List)):
        value = np.flip(falsePosFracArr, axis=0)[i, j]
        #plt.annotate(custom_format(value), (j, i), color='red', ha='center', va='center')
plt.show()
plt.xticks(rotation = 45) 
plt.colorbar()

In [ ]:
lowPercent      = .87
highPercent     = .99

arrMask     = np.logical_and(truePosFracArr > lowPercent, truePosFracArr < highPercent)
arrMask[3,1] = True
arrMask[4,2] = True
arrMask[8,5] = True

print(arrMask.sum())
plt.figure()
plt.imshow(np.flip(arrMask, axis=1))
plt.show()

In [ ]:
for nuIdx in enumerate(nu0List):
    for powIdx, sigPow in enumerate(sigPowList):
        print(f'nu0, sigPow :  {nu0, sigPow}' )

        

In [ ]:
for i in range(arrMask.shape[0]):
    for j in range(arrMask.shape[1]):
        if arrMask[i, j]:
            print(f"({nu0List[j]}, {sigPowList[i]})")

In [ ]:
n = 10
new_shape = (n * len(nu0List), n * len(sigPowList))
new_nu0List = np.zeros(new_shape[1])
new_sigPowList = np.zeros(new_shape[0])
new_array = np.zeros(new_shape)

trial_params = {
    'medFiltBins': medFiltBins,
    'butterFreq': butterFreq,
    'butterOrder': butterOrder,
    'freqStart': startFreq,
    'freqStop': stopFreq,
    'percent': percent,
    'overlap': overlap,
    'CL': CL
}


for i in range(arrMask.shape[0]):
    for j in range(arrMask.shape[1]):
        if arrMask[i, j]:
            # Subsampling case: generate finer grid and run analysis function
            for k in range(n):
                for l in range(n):
                    new_nu0 = nu0List[j] + (k / (n - 1)) * (nu0List[j + 1] - nu0List[j]) if j + 1 < len(nu0List) else nu0List[j]
                    new_sigPow = sigPowList[i] + (l / (n - 1)) * (sigPowList[i + 1] - sigPowList[i]) if i + 1 < len(sigPowList) else sigPowList[i]
                    new_i = i * n + k
                    new_j = j * n + l
                    new_nu0List[new_j] = new_nu0
                    new_sigPowList[new_i] = new_sigPow

                    print(f'i, j, k, l: {i, j, k, l}')

                    print(f'nu0, sigPow :  {new_nu0, new_sigPow}' )
                    #make more density around diagional
                    #nTrials = int((1-(abs(nuIdx-powIdx)/10)) * nTrailsPerStep) + 1
                    #print(nuIdx,powIdx)
                    #print(nTrials)
                    nTrials = nTrailsPerStep
                    '''with ThreadPoolExecutor(max_workers=40) as executor:

                        results = [executor.submit(run_trial, nu0, sigPow, trial_params) for _ in range(nTrials)]
                        truePosCount = sum(result.result()[0] for result in results)
                        falsePosCount = sum(result.result()[1] for result in results)

                        truePosFrac = truePosCount/nTrials
                        falsePosFrac = falsePosCount/nTrials'''
                    new_array[new_i, new_j] =  .43 #truePosFrac # Call to your specific analysis function
        else:
            # No subsampling case: Fill all n squares with the original value
            for k in range(n):
                for l in range(n):
                    new_i = i * n + k
                    new_j = j * n + l
                    new_nu0List[new_j] = nu0List[j]
                    new_sigPowList[new_i] = sigPowList[i]
                    new_array[new_i, new_j] = truePosFracArr[i, j]

plt.figure()
plt.imshow(new_array)
